In [ ]:
# Woolworth Deutschland - Darts - Monthly Forecast 3 years ahead - Monthly Turnover in Mio EUR

# Forecasting methods: ARIMA, SARIMA, Dynamic regression models, hyerarchical forecasting (top down, bottom up),
# RNN, MLP, XGBoost, CCN, Regression Forests, LSTM, ETS, TBATS, DeepAR, neural networks, bootstrapping, bagging,
# vector autoregressions, Markov Chains, RBFF, TFT, TCN, GRU, ADIDA, Croston, MAPA,  ...
# libraries: pmdarima, prophet, sktime, Darts, GluonTS, PyTorch,...

# Darts - automated framework to execute predictions Exponential Smoothing, Arima, Prophet, RNN,Theta, Naive Bayes, etc.
# Darts operates with monthly numbers.

# 1. Resample Annual to Monthly ( we do not have monthly data, use e.g. ffill method)
# 2. Execute, fit and compare forecast models

In [ ]:
# import libraries
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_colwidth', 200)
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode()
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
# 1. Resample from yearly to monthly ( upsampling)

In [ ]:
# Load the data
df = pd.read_csv('Woolworth_2.csv', parse_dates=True)

In [ ]:
# keep the original for later calculations
df1 =df

In [ ]:
df1.dtypes

In [ ]:
#data

In [ ]:
# Convert string column to datetime
df1['Year'] = pd.to_datetime(df1['Year'])
# Set time column as index
df1 = df1.set_index('Year')
df1

In [ ]:
df3 = df1['Umsatz Woolworth DE']

In [ ]:
# prepare the empty dataframe for monthly values
df_month = df1.asfreq('ME')
#df_month

In [ ]:
# upscaling - use e.g. ffill (forward fill method)
df_mean_temp_Y = df1[['Umsatz Woolworth DE']]
df_mean_temp_M= df_mean_temp_Y.asfreq('ME')
df_mean_temp_M['ffill'] = df_mean_temp_Y.asfreq('ME', method='ffill')
#df_mean_temp_M

In [ ]:
# resampled
resample = df_mean_temp_M['ffill']
resample

In [ ]:
# save monthly data to csv
resample.to_csv('Woolworth_resampled.csv', index=True)

In [ ]:
# plot the target variable series y
ax = resample.plot(figsize=(12, 4), color=['black'])
ax.set_ylabel('Woolworth DE - Umsatz Mio EUR')
ax.set_xlabel('Year')
ax.set_title('Monthly Resampled Data - ffill')

plt.show()

In [ ]:
# 2. Darts

In [ ]:
pip install darts

In [ ]:
from darts import TimeSeries

In [ ]:
from darts import models

In [ ]:
from darts.models import ExponentialSmoothing, ARIMA, Prophet, RNNModel, NaiveDrift, Theta

In [ ]:
from darts.metrics import mape, mase, mae, mse, ope, r2_score, rmse, rmsle

In [ ]:
from darts.dataprocessing.transformers import Scaler

In [ ]:
# Load the da ta
rf = pd.read_csv('Woolworth_resampled.csv', parse_dates=True)

In [ ]:
rf['Year'] = pd.to_datetime(rf['Year'])
rf

In [ ]:
rf.dtypes

In [ ]:
# in the case you wish to use train and val split to compare RMSE, MAE, R2, ...
# from sklearn.model_selection import train_test_split
# train, val = train_test_split(rf, test_size=0.2, shuffle=False, random_state=None)

In [ ]:
scaler=Scaler()

In [ ]:
ts=TimeSeries.from_dataframe(rf, time_col="Year",value_cols="ffill")

In [ ]:
ts_scaled=scaler.fit_transform(ts)

In [ ]:
ts_scaled.plot("Actual Data",figsize=(12, 4), color=['black'])

In [ ]:
# 2.1. Darts - Exponential Smoothing
modelE = ExponentialSmoothing()
modelE.fit(ts_scaled)
forecastE = modelE.predict(36)
forecastE.plot(label="Exponential Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# 2.2. Darts - ARIMA

In [ ]:
modelARIMA=ARIMA()
modelARIMA.fit(ts_scaled)
forecastARIMA = modelARIMA.predict(36)
forecastARIMA.plot(label="ARIMA Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# 2.3. Darts - Prophet

In [ ]:
modelPROPHET=Prophet()
modelPROPHET.fit(ts_scaled)
forecastPROPHET = modelPROPHET.predict(36)
forecastPROPHET.plot(label="PROPHET Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# 2.4. Darts - RNN

In [ ]:
modelRNN=RNNModel(model="LSTM", input_chunk_length=20, output_chunk_length=20,n_epochs=50)
modelRNN.fit(ts_scaled)
forecastRNN = modelRNN.predict(36)
forecastRNN.plot(label="RNN Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# 2.5. Darts - Naive Bayes

In [ ]:
modelNaive=NaiveDrift()
modelNaive.fit(ts_scaled)
forecastNaive = modelNaive.predict(36)
forecastNaive.plot(label="Naive Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# 2.6. Darts - Theta

In [ ]:
modelTheta=Theta()
modelTheta.fit(ts_scaled)
forecastTheta = modelTheta.predict(36)
forecastTheta.plot(label="Theta Prediction",figsize=(12, 3), color=['black'])
ts_scaled.plot("Actual Data",figsize=(12, 3), color=['black'])

In [ ]:
# https://arxiv.org/abs/2404.12814
# https://unit8co.github.io/darts/
# https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.fbprophet.Prophet.html
# https://www.handelsdaten.de/
# https://de.statista.com/
# https://pypi.org/project/prophet/
# https://magnimindacademy.com/blog/time-series-forecasting-with-darts-a-hands-on-tutorial/
# https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo
# https://www.analyticsvidhya.com/blog/2022/06/time-series-forecasting-using-python/
# https://towardsdatascience.com/darts-swiss-knife-for-time-series-forecasting-in-python-f37bb74c126/
# https://en.wikipedia.org/wiki/Metropolis-adjusted_Langevin_algorithm
# https://docs.oracle.com/cd/E57185_01/CBREG/ch06s02s02s01.html
# https://www.kaggle.com/code/jurk06/auto-arima-on-multivariate-time-series
# https://www.kaggle.com/code/prashant111/tutorial-time-series-forecasting-with-prophet
# https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c
# https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/